In [1]:
# Init
import os
import sys

module_path = os.path.abspath(os.path.join("../src/simulicronalpha/"))
if module_path not in sys.path:
    sys.path.append(module_path)

# Imports
import numpy as np
import pandas as pd
import warnings
import multiprocessing
np.set_printoptions(suppress=True)
from numpy import concatenate as c
from numpy import cumsum
import random
%load_ext line_profiler

In [2]:
from generateSim import (generatePopulation, generateGenome)
from stats import stats
from regulation import regulation
from checkCopyNumber import checkCopyNumber
from transposition import transposition
#from recombination import recombination

In [3]:
from popSim import runBatch, runSim

In [6]:
gen,piset,piIndice, rates = generateGenome(
    numberOfInsertionSites=1000,
    numberOfChromosomes=6,
    baseRecombinationRate=0.1,
    baseSelection=0,
    baseTau=1,
)
pop, tr , TEset= generatePopulation(
    gen,
    piIndice,
    NumberOfIndividual=100,
    NumberOfTransposonTypes=1,
    NumberOfInsertionsPerType=[1],
    FrequencyOfInsertions=[1.0],
    ExcisionRates = [1.0]
)

In [7]:
%lprun -f runSim runSim(gen,pop,tr,TEset,1,200,rates,piset)

Timer unit: 1e-06 s

Total time: 40.477 s
File: /home/siddharth/Documents/Projects/Simulicron/src/simulicronalpha/popSim.py
Function: runSim at line 25

Line #      Hits         Time  Per Hit   % Time  Line Contents
    25                                           def runSim(
    26                                               genomeMatrix,
    27                                               populationMatrix,
    28                                               transposonMatrix,
    29                                               TEset,
    30                                               NumberOfTransposonInsertions,
    31                                               generations,
    32                                               genMap,
    33                                               piSet,
    34                                           ):
    35                                               # ------------------#
    36                                               # la

In [ ]:
runSim(gen,pop,tr,TEset,1,100,rates,piset)

In [ ]:
%lprun -f recombination recombination(rates=rates, transposonMatrix=tr, v1=[52], v2=[65])

In [ ]:
%lprun -f runBatch runBatch(numberOfSimulations=1,baseSelection=0,baseInsertionProb=1,numberOfInsertionSites=10000,numberOfChromosomes=6,baseRecombinationRate=0.01,baseTau=1,numberOfPiRNA=6,piPercentage=3,enablePiRecombination=False,NumberOfIndividual=1000,NumberOfTransposonTypes=1,NumberOfInsertionsPerType=[1],FrequencyOfInsertions=[1.0],ExcisionRates=[1.0],RepairRates=[1],InsertionRates=[1],HardyWeinberg=False,NumberOfGenerations=10,numberOfThreads=1,)

In [ ]:
np.set_printoptions(suppress=True,)

In [ ]:
%lprun -f transposition transposition(transposonMatrix=tr, genomeMatrix=gen, NumberOfTransposonInsertions=1, TEset=TEset, v1=[52,101,321,433,12,23,234,123,657,29], v2=[65,54,22,121,23])

In [ ]:
piIndice[1]

In [ ]:
piset

In [ ]:
list(range(1,4))

In [ ]:
piCoord = []
for i in piset.values():
    piCoord = piCoord + (list(range(i[0],i[1])))

In [ ]:
(tr[[2,3],3] * 0).tolist()

In [ ]:
list(TEset.keys())

In [ ]:
for i in TEset.keys():
    print (i)

In [ ]:
len(TEset.keys())

In [ ]:
tr

In [ ]:
k = [0.0,1.0,1.0,1.0,1.0,1.0]

In [ ]:
np.append(tr, np.array([k]), axis=0)

In [ ]:
tr[:,3]

In [ ]:
x = tr

In [ ]:
tID = [int(i) for i in x[:,0].tolist()]
tpos = [int(i) for i in x[:,1].tolist()]
tval = [float(i) for i in x[:,3].tolist()]

In [ ]:
k = np.array([tID,tpos,tval], dtype="object").T
z = [1, 5451, 1.0]
l = k

In [ ]:
np.vstack((tID, tpos, tval), dtype="object").T

In [ ]:
%%timeit
l=k
for i in range(1000):
    l = np.append(k, l, axis=0)

In [ ]:
l = np.zeros((10000000,6), dtype=object)

In [ ]:
sys.getsizeof(l)

In [ ]:
len(l)

In [ ]:
tr

In [ ]:
np.append(tr, np.zeros((10000000,6), dtype=object), axis=0)

In [ ]:
type (l[1,0])

In [ ]:
l[0,0] = 5.5

In [ ]:
c = 0
c += 1

In [ ]:
c += 1

In [ ]:
c

In [ ]:
tr[[1,2,3],0]

In [ ]:
listA = [0]
listB = listA
listB.append(1)
print (listA)

In [ ]:
l = [5,4,2]

In [ ]:
tr[l,0]